# Implementação de Localização Via QR Code

# Módulos Necessários

In [ ]:
!pip install opencv-python pyzbar pygame

In [ ]:
!pip install pillow pyzbar pygame

## Gerador de QR Code

O código a seguir cria um QR code com o nome do audio que irá rodar. Para tanto salve os audios com o nome que irá colocar nos QR codes. Por exemplo, para abrir o "audio_1" execute o código a seguir e coloque "audio_1".

In [ ]:
import qrcode

def gerar_qrcode(nome):
    """
    Gera um QR Code com o mesmo nome no conteúdo e no arquivo de saída.
    Formato: 'audio_x' (ex: audio_1)
    """
    # Garante que o nome termina com .png
    nome_arquivo = f"{nome}.png"
    
    # Cria o QR Code
    qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_L,
        box_size=10,
        border=4,
    )
    qr.add_data(nome)  # Usa o mesmo nome como conteúdo
    qr.make(fit=True)
    
    # Salva a imagem
    img = qr.make_image(fill_color="black", back_color="white")
    img.save(nome_arquivo)
    
    print(f"QR Code gerado: '{nome_arquivo}' (Contém: '{nome}')")

if __name__ == "__main__":
    nome = input("Digite o nome no formato 'audio_x' (ex: audio_1): ").strip()
    if not nome.startswith('audio_'):
        print("⚠️O nome deve começar com 'audio_'")
    else:
        gerar_qrcode(nome)

## Programa de Abertura da Webcam e Captura Para Captura do QR Code

Os audios a serem reproduzidos devem estar na pasta "audios".

In [ ]:
import cv2
from pyzbar import pyzbar
import pygame
import os
import re
import time

# Configuration
AUDIO_FOLDER = "audios"
CAMERA_INDEX = 2
AUDIO_DELAY = 3  # 3 second delay between same audio plays

# Initialize audio mixer
pygame.mixer.init()

def setup_camera(index=0):
    """Initialize and return camera capture object"""
    camera = cv2.VideoCapture(index)
    if camera.isOpened():
        print(f"✅ Camera {index} connected (Iriun Webcam)")
        return camera
    print(f"❌ Camera {index} not found. Trying next index...")
    return setup_camera(index + 1)

def clean_filename(text):
    """Sanitize filename and ensure .mp3 extension"""
    clean = re.sub(r'[^a-zA-Z0-9_.]', '', text)
    if not clean.lower().endswith('.mp3'):
        clean += '.mp3'
    return clean.lower()

def find_audio_file(qr_text, folder):
    """Locate audio file matching QR code content"""
    filename = clean_filename(qr_text)
    print(f"Searching for: '{filename}'")
    
    for file in os.listdir(folder):
        if file.lower() == filename:
            return os.path.join(folder, file)
    return None

# Main execution
def main():
    camera = setup_camera(CAMERA_INDEX)
    
    if not os.path.exists(AUDIO_FOLDER):
        print(f"⚠️ Folder '{AUDIO_FOLDER}' not found!")
        return

    print("🔍 Point camera at QR code...")
    
    last_audio = None
    last_play_time = 0

    while True:
        success, frame = camera.read()
        if not success:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        codes = pyzbar.decode(gray)
        
        for code in codes:
            text = code.data.decode("utf-8").strip()
            print(f"QR detected: '{text}'")
            
            audio_path = find_audio_file(text, AUDIO_FOLDER)
            
            if audio_path:
                current_time = time.time()
                if (audio_path != last_audio or 
                    (current_time - last_play_time) >= AUDIO_DELAY):
                    try:
                        pygame.mixer.music.load(audio_path)
                        pygame.mixer.music.play()
                        print(f"🎵 Playing: {os.path.basename(audio_path)}")
                        last_audio = audio_path
                        last_play_time = current_time
                        while pygame.mixer.music.get_busy():
                            cv2.waitKey(100)
                    except Exception as e:
                        print(f"Audio error: {e}")
                else:
                    print(f"⏳ Waiting {AUDIO_DELAY}s to repeat audio...")
            else:
                print(f"⚠️ Audio not found for: '{text}'")
                print(f"Available files: {os.listdir(AUDIO_FOLDER)}")
            
            # Draw rectangle around QR code
            x, y, w, h = code.rect
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
        cv2.imshow("QR Code Audio Player", frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    camera.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()